In [ ]:
# Clone the Wav2Lip repository
!git clone https://github.com/indianajson/wav2lip-HD.git
basePath = "/content/wav2lip-HD"
%cd {basePath}

wav2lipFolderName = 'Wav2Lip-master'
wav2lipPath = basePath + '/' + wav2lipFolderName

# Download the required pre-trained models for Wav2Lip
!wget 'https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth' -O {wav2lipPath}'/s3fd-619a316812.pth'

!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O {wav2lipPath}'/checkpoints/wav2lip_gan.pth'

!gdown https://drive.google.com/uc?id=1fQtBSYEyuai9MjBOF8j7zZ4oQ9W2N64q --output {wav2lipPath}'/checkpoints/wav2lip_gan.pth'

# Install dependencies
!pip install -r requirements.txt
!pip install -U librosa==0.8.1 # The process will fail without downgrading librosa

# Create inputs directory
!mkdir inputs

from IPython.display import clear_output
clear_output()

print("Installation complete.")


Installation complete.


In [ ]:
import os
outputPath = basePath+'/outputs'
inputAudio = 'dhoni_translated.mp3'
inputAudioPath = basePath + '/inputs/' + inputAudio
inputVideo = 'DhoniTest.mov'
inputVideoPath = basePath + '/inputs/'+inputVideo
lipSyncedOutputPath = basePath + '/outputs/result.mp4'
model = "wav2lip_gan"

if not os.path.exists(outputPath):
  os.makedirs(outputPath)


!cd $wav2lipFolderName && python inference.py \
--checkpoint_path checkpoints/{model}.pth \
--face {inputVideoPath} \
--audio {inputAudioPath} \
--outfile {lipSyncedOutputPath}


In [ ]:
!pip install opencv-python-headless numpy
import cv2
import numpy as np

def extract_histograms(video_path):
    video_cap = cv2.VideoCapture(video_path)
    histograms = []

    while True:
        ret, frame = video_cap.read()
        if not ret:
            break

        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        hist = cv2.calcHist([hsv_frame], [0, 1, 2], None, [50, 60, 60], [0, 180, 0, 256, 0, 256])
        cv2.normalize(hist, hist, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
        histograms.append(hist.flatten())

    video_cap.release()
    return histograms

def calculate_similarity(histograms1, histograms2):
    min_length = min(len(histograms1), len(histograms2))
    score = 0

    for i in range(min_length):
        score += cv2.compareHist(histograms1[i], histograms2[i], cv2.HISTCMP_CORREL)

    average_score = score / min_length if min_length > 0 else 0
    return average_score

video_path1 = '/content/Dhoni.mp4' #input
video_path2 = '/content/Dhoni_resulty.mp4' #result

histograms1 = extract_histograms(video_path1)
histograms2 = extract_histograms(video_path2)

similarity_score = calculate_similarity(histograms1, histograms2)
print(f"Average Similarity Score: {similarity_score:.4f}")